In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from ridge_utils.DataSequence import DataSequence
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
import sasc.viz
import joblib
import dvu
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import brain_drive_resps_dir
from neuro import analyze_helper, viz
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
flatmaps_per_question = __import__('06_flatmaps_per_question')

### load average flatmaps per explanation braindrive

In [ ]:
brain_drive_avgs_pilot = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot.pkl'))
brain_drive_avgs_pilot5 = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot5.pkl'))
brain_drive_avgs = brain_drive_avgs_pilot | brain_drive_avgs_pilot5

### load weights for jointly fitted 35-question model

In [ ]:
# # load questions model weights
# results_dir = analyze_helper.best_results_dir
# rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)
# metric_sort = 'corrs_tune_pc_weighted_mean'

# # pick model to interpret
# r = rr
# r = r[r.qa_questions_version == 'v3_boostexamples_merged']
# r = r[r.num_stories == -1]
# r = r[r.weight_enet_mask_num_nonzero == 35]
# r = r[r.feature_space == 'qa_embedder']
# cols_varied = [c for c in cols_varied if not c in ['num_stories',
#                                                    'feature_selection_alpha', 'feature_selection_stability_seeds']]
# args0 = r[r.subject == 'S02'].iloc[0]
# weights, weights_pc = flatmaps_per_question.get_weights_top(args0)
# qs_selected = questions[args0['weight_enet_mask']]
# df_w_selected35 = pd.DataFrame({'question': qs_selected, 'weights': [
#                     w for w in weights]}).set_index('question')
# joblib.dump((args0, qs_selected, df_w_selected35),
# '../qa_results/processed/selected_weights.pkl')

args0, qs_selected, df_w_selected35 = joblib.load(
    '../qa_results/processed/selected_weights.pkl')

### load weights for individual question models

In [ ]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/jun16_gpt4'
rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)
r = rr
r = r[r.subject == 'S02']
r = r[r.use_added_wordrate_feature == 0]
r = r[r.feature_space == 'qa_embedder']

weights = np.array([
    flatmaps_per_question.get_weights_top(r.iloc[i])[0]
    for i in tqdm(range(len(r)))
]).squeeze()
qs_selected = r['qa_questions_version']
df_w_individual = pd.DataFrame({'question': qs_selected, 'weights': [
    w for w in weights]}).set_index('question')
joblib.dump((r, qs_selected, df_w_individual),
            '../qa_results/processed/individual_weights.pkl')
r, qs_selected, df_w_individual = joblib.load(
    '../qa_results/processed/individual_weights.pkl')

### load weights for individual question models **with wordrate**

In [ ]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/jun16_gpt4'
rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)
r = rr
r = r[r.subject == 'S02']
r = r[r.use_added_wordrate_feature == 1]
r = r[r.feature_space == 'qa_embedder']

weights = np.array([
    flatmaps_per_question.get_weights_top(r.iloc[i])[0]
    for i in tqdm(range(len(r)))
])[:, 0, :].squeeze()
qs_selected = r['qa_questions_version']
df_w_individual = pd.DataFrame({'question': qs_selected, 'weights': [
    w for w in weights]}).set_index('question')
joblib.dump((r, qs_selected, df_w_individual),
            '../qa_results/processed/individual_weights_wordrate.pkl')
r, qs_selected, df_w_individual_wordrate = joblib.load(
    '../qa_results/processed/individual_weights_wordrate.pkl')

### Make comparison

In [ ]:
# matches
qa_list = [
    # approx matches
    'Is time mentioned in the input?',
    'Does the input contain a measurement?',
    # 'Does the input contain a number?',
    'Does the sentence mention a specific location?',
    # 'Does the sentence describe a relationship between people?',
    # 'Does the sentence describe a relationship between people?',
    'Does the text describe a mode of communication?',
    # 'Does the sentence contain a negation?',
]
bd_list = [
    # approx matches
    ('time', 212),
    ('measurements', 171),
    # ('measurements', 171),
    # ('moments',	337),
    # ('locations', 122),
    ('locations', 368),
    # ('emotion', 179),
    # ('emotional expression', 398),
    ('communication', 299),
    # ('negativity', 248)
]


qa_list += [
    'Is the sentence abstract rather than concrete?',
    'Does the sentence contain a cultural reference?',
    'Does the sentence include dialogue?',
    'Is the input related to a specific industry or profession?',
    'Does the sentence contain a negation?',
    'Does the input contain a number?',
    "Does the sentence express the narrator's opinion or judgment about an event or character?",
    'Does the sentence describe a personal or social interaction that leads to a change or revelation?',
    'Does the sentence describe a personal reflection or thought?',
    'Does the sentence involve an expression of personal values or beliefs?',
    'Does the sentence describe a physical action?',
    'Does the input involve planning or organizing?',
    'Does the sentence contain a proper noun?',
    'Does the sentence describe a relationship between people?',
    'Does the sentence describe a sensory experience?',
    'Does the sentence involve the mention of a specific object or item?',
    'Does the sentence include technical or specialized terminology?',
]

bd_list += [
    ('abstract descriptions', 'qa'),
    ('cultural references', 'qa'),
    ('dialogue', 'qa'),
    ('industry or profession', 'qa'),
    ('negations', 'qa'),
    ('numbers', 'qa'),
    ('opinions or judgments', 'qa'),
    ('personal or interactions interactions', 'qa'),
    ('personal reflections or thoughts', 'qa'),
    ('personal values or beliefs', 'qa'),
    ('physical actions', 'qa'),
    ('planning or organizing', 'qa'),
    ('proper nouns', 'qa'),
    ('relationships between people', 'qa'),
    ('sensory experiences', 'qa'),
    ('specific objects or items', 'qa'),
    ('technical or specialized terminology', 'qa')]

df_pairs = pd.DataFrame({
    'qa_weight': qa_list,
    'bd_avg_resp': bd_list,
})


df_qa_weights = df_w_individual
df_qa_weights = df_w_individual_wordrate
df_qa_weights = df_w_individual

# remove columns that aren't known
df_pairs = df_pairs[df_pairs['qa_weight'].isin(df_qa_weights.index)]

wt_qas = df_qa_weights.loc[df_pairs['qa_weight'].values]['weights'].values
wt_bds = [brain_drive_avgs[bd] for bd in df_pairs['bd_avg_resp'].values]

# mask = args0['corrs_test'] >= 0
# wt_qas = [wt_qa[mask] for wt_qa in wt_qas]
# wt_bds = [wt_bd[mask] for wt_bd in wt_bds]

In [ ]:
corrs = pd.DataFrame(
    np.zeros((len(df_pairs), len(df_pairs))),
    columns=df_pairs['bd_avg_resp'].astype(str),
    # [f'{bd_list[i][0]}_{bd_list[i][1]}'.replace('_qa', '') for i in range(len(bd_list))],
    # index=[analyze_helper.abbrev_question(q) for q in qa_list],
    index=df_pairs['qa_weight'].astype(str),
)
for i, qa in enumerate(df_pairs['qa_weight']):
    for j, bd in enumerate(df_pairs['bd_avg_resp']):
        corrs.iloc[i, j] = np.corrcoef(wt_qas[i], wt_bds[j])[0, 1]


# normalize each column
# corrs = corrs / corrs.abs().max()
# normalize each row to mean zero stddev 1
# corrs = (corrs - corrs.mean()) / corrs.std()
# plt.figure(figsize=(20, 10))
vmax = np.max(np.abs(corrs.values))
# sns.clustermap(corrs, annot=False, cmap='RdBu', vmin=-vmax, vmax=vmax, dendrogram_ratio=0.01)

sns.heatmap(corrs, annot=False, cmap='RdBu_r', vmin=-vmax, vmax=vmax)
# plt.imshow(corrs, cmap='RdBu_r', vmin=-vmax, vmax=vmax)
# plt.xticks(range(len(bd_list)), [f'{bd[0]}' for bd in bd_list], rotation=90)
# plt.yticks(range(len(qa_list)), qa_list)
# plt.colorbar()
dvu.outline_diagonal(corrs.values.shape, roffset=0.5, coffset=0.5)
plt.show()

# barplot of diagonal
sns.barplot(y=corrs.columns, x=np.diag(corrs), color='gray')
plt.axvline(np.diag(corrs).mean())
print('mean', np.diag(corrs).mean())
plt.show()

In [ ]:
# quickshow(wt_qas[0])
# quickshow(wt_bds[0])
sasc.viz.quickshow(
    wt_qas[0],
    subject="UTS02",
)

In [ ]:
wt_qas[0].shape